In [3]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# movies_data = pd.read_csv("/kaggle/input/the-movies-dataset/movies_metadata.csv")

movies_data = pd.read_csv("/kaggle/input/movies-csv/movies.csv")

In [5]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [6]:
movies_data.tail()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,"[{'name': 'Daniel Hsia', 'gender': 2, 'departm...",Daniel Hsia
4802,4802,0,Documentary,NaN,25975,obsession camcorder crush dream girl,en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16,Drew Barrymore Brian Herzlinger Corey Feldman ...,"[{'name': 'Clark Peterson', 'gender': 2, 'depa...",Brian Herzlinger


In [7]:
#movies data shape 
movies_data.shape

(4803, 24)

In [8]:
selected_features=['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [10]:
for feature in selected_features:
    movies_data[feature]=movies_data[feature].fillna('')
    

In [11]:
combined_features=movies_data['genres']+movies_data['keywords']+movies_data['tagline']+movies_data['cast']+movies_data['director']

In [12]:
print(combined_features)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object


In [14]:
#converting textual data to numerical values
vectorizer=TfidfVectorizer()
feature_vectors=vectorizer.fit_transform(combined_features)
print (feature_vectors)

  (0, 3583)	0.16941894714909375
  (0, 20729)	0.2738578969459924
  (0, 16614)	0.15696073877453268
  (0, 14550)	0.2236809363648068
  (0, 23267)	0.16128139780622516
  (0, 26547)	0.19638671147741732
  (0, 22464)	0.20207089568711745
  (0, 21487)	0.21550128478931546
  (0, 27540)	0.19771357974524176
  (0, 27182)	0.23480088356130552
  (0, 21521)	0.1562356665943368
  (0, 18558)	0.2612168335104853
  (0, 18164)	0.08690831799482265
  (0, 27161)	0.1261988657937151
  (0, 24197)	0.07518543993419265
  (0, 22778)	0.2738578969459924
  (0, 4676)	0.2452909770424961
  (0, 26334)	0.13059723071916296
  (0, 22916)	0.33668756406929184
  (0, 10193)	0.16532432420812987
  (0, 4456)	0.21799675215510664
  (0, 9304)	0.2738578969459924
  (0, 21836)	0.09966592997173944
  (0, 8936)	0.11806131645084653
  (0, 444)	0.09109355212252294
  :	:
  (4801, 5449)	0.31510541707008866
  (4801, 22170)	0.31510541707008866
  (4801, 27361)	0.31510541707008866
  (4801, 27513)	0.3005604008026175
  (4801, 8121)	0.2611505149930138
  (4801,

In [15]:
similarity=cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.06865296 0.01492221 ... 0.         0.         0.        ]
 [0.06865296 1.         0.02799128 ... 0.01243107 0.         0.        ]
 [0.01492221 0.02799128 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01243107 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [16]:
print(similarity.shape)

(4803, 4803)


In [63]:
movie_name=input("Enter your favourite movie name")


Enter your favourite movie name home alone


In [64]:
list_of_all_titles=movies_data['title'].tolist()
print(list_of_all_titles[:5])

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter']


In [65]:
find_close_match=difflib.get_close_matches(movie_name,list_of_all_titles)
print(find_close_match)

['Home Alone', 'The Salon', 'Tom Jones']


In [66]:
close_match=find_close_match[0]
print(close_match)

Home Alone


In [67]:
index_of_the_movie=movies_data[movies_data.title==close_match]['index'].values[0]
print(index_of_the_movie)

2511


In [68]:
similarity_score=list(enumerate(similarity[index_of_the_movie]))
print(similarity_score[:10])



[(0, 0.005757462200804104), (1, 0.01510085874790052), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.0)]


In [69]:
print(len(similarity_score))

4803


In [70]:
sorted_similar_movies=sorted(similarity_score,key=lambda x:x[1], reverse=True)
print(sorted_similar_movies[:10])

[(2511, 1.0000000000000002), (2086, 0.34837893085754656), (3670, 0.23021411026059344), (636, 0.12610969766204863), (4284, 0.12576820407473627), (4689, 0.12376664622694528), (3614, 0.12248449774262397), (3639, 0.1121924525733501), (1214, 0.11126661211203152), (2745, 0.10976422469606721)]


In [92]:
def similar_movies(movie_name):
    list_of_all_titles=movies_data['title'].tolist()
    
    find_close_match=difflib.get_close_matches(movie_name,list_of_all_titles)
    if(len(find_close_match)==0):
        return 0
    index_of_the_movie=movies_data[movies_data.title==find_close_match[0]]['index'].values[0]
    similarity_score=list(enumerate(similarity[index_of_the_movie]))
    sorted_similar_movies=sorted(similarity_score,key=lambda x:x[1], reverse=True)
    return sorted_similar_movies

In [97]:

def testwithinput():
    name=input ("Enter your favourite movie name:")
    sorted_similar_movies=similar_movies(name)
    if (sorted_similar_movies==0):
         print("No movie found")

    else:
        print("Suggested Movies for you:")
        i=1
        for movie in sorted_similar_movies:
            index=movie[0]
            title_from_index=movies_data[movies_data.index==index]["title"].values[0]
            if(i<30):
                print (i, ".",title_from_index)
                i+=1

        
      
    



        

In [99]:
testwithinput()

Enter your favourite movie name: home alone


Suggested Movies for you:
1 . Home Alone
2 . Home Alone 2: Lost in New York
3 . Running Forever
4 . The Santa Clause 2
5 . C.H.U.D.
6 . The Trials Of Darryl Hunt
7 . Saved!
8 . Party Monster
9 . Ri¢hie Ri¢h
10 . This Christmas
11 . 8 Heads in a Duffel Bag
12 . Love the Coopers
13 . The Croods
14 . Lethal Weapon 4
15 . Eulogy
16 . The Hitchhiker's Guide to the Galaxy
17 . Entrapment
18 . Raging Bull
19 . GoodFellas
20 . Animals
21 . The Stepfather
22 . Harry Potter and the Philosopher's Stone
23 . Casino
24 . Hostage
25 . The Grand Budapest Hotel
26 . Slacker
27 . The Last Song
28 . Lethal Weapon 3
29 . My Girl


In [102]:
testwithinput()

Enter your favourite movie name: spider man


Suggested Movies for you:
1 . Spider-Man
2 . Spider-Man 3
3 . Spider-Man 2
4 . The Notebook
5 . Seabiscuit
6 . Horrible Bosses
7 . Finding Nemo
8 . The Good German
9 . Clear and Present Danger
10 . The Count of Monte Cristo
11 . Labor Day
12 . Drag Me to Hell
13 . Oz: The Great and Powerful
14 . Drop Dead Gorgeous
15 . Velvet Goldmine
16 . Wonder Boys
17 . The Legend of Bagger Vance
18 . The Ice Storm
19 . The Queen
20 . The Virgin Suicides
21 . Elizabethtown
22 . Bambi
23 . Brothers
24 . Wimbledon
25 . Adam Resurrected
26 . The Great Gatsby
27 . Interview with the Vampire
28 . Small Soldiers
29 . Midnight Special


In [103]:
testwithinput()

Enter your favourite movie name: toy story


Suggested Movies for you:
1 . Toy Story
2 . Toy Story 2
3 . Toy Story 3
4 . Quest for Camelot
5 . Cloud Atlas
6 . Cars 2
7 . Small Soldiers
8 . Jingle All the Way
9 . Sleeper
10 . Big
11 . Cars
12 . Larry Crowne
13 . Fantastic Mr. Fox
14 . Treading Water
15 . Monster House
16 . Joe Somebody
17 . Spun
18 . Valiant
19 . The Curse of the Jade Scorpion
20 . I Love You, Don't Touch Me!
21 . Christmas with the Kranks
22 . Galaxy Quest
23 . That Thing You Do!
24 . Saving Mr. Banks
25 . The Smurfs
26 . Radio Days
27 . Vamps
28 . Admission
29 . To Rome with Love


In [105]:
testwithinput()

Enter your favourite movie name: mission impossible ghost protocol


Suggested Movies for you:
1 . Mission: Impossible - Ghost Protocol
2 . Mission: Impossible - Rogue Nation
3 . Spectre
4 . John Q
5 . Containment
6 . Phone Booth
7 . The Net
8 . Kill the Messenger
9 . The Expendables 3
10 . The Incredibles
11 . When the Cat's Away
12 . Heavenly Creatures
13 . Haywire
14 . Doubt
15 . Star Trek Beyond
16 . Ravenous
17 . The Hurt Locker
18 . Gangster Squad
19 . Clerks II
20 . Baggage Claim
21 . Warcraft
22 . Daylight
23 . 2 Guns
24 . Unaccompanied Minors
25 . Zambezia
26 . Mission: Impossible III
27 . The Punisher
28 . Hot Fuzz
29 . The Immigrant
